<a href="https://colab.research.google.com/github/carolinezep/Teste/blob/master/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
#!pip install google-genai
#!pip install --upgrade google-generativeai
import sqlite3
import os

from google.colab import userdata
#userdata.get('GOOGLE_API_KEY_1')

os.environ['GOOGLE_API_KEY_1'] = userdata.get('GOOGLE_API_KEY_1')
os.environ['cliente_id'] = userdata.get('cliente_id')


In [59]:
from google import genai

# Get the API key from the environment variable
api_key = os.getenv('GOOGLE_API_KEY_1')
# Get the API key from the environment variable
api_key_2 = os.getenv('cliente_id')

# Initialize the client with the API key
client = genai.Client(api_key=api_key)

from google.genai import types
modelo = "gemini-2.0-flash"

chat_config = types.GenerateContentConfig(
    system_instruction = "Você é um assistente pessoal que lida com pais e maes de bebe e que é especialista na área e você sempre responde de forma sucinta.",
)
chat = client.chats.create(model=modelo, config=chat_config)

In [156]:
# Instala as bibliotecas necessárias
# Certifique-se de executar esta célula no início do seu notebook
#!pip install --upgrade google-auth-oauthlib
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#!pip install pandas
from IPython.display import display, HTML
import google.auth
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import datetime
import os
import json
import google.auth
import google.colab
import pandas as pd # Import pandas at the top level

# Importa a biblioteca para acessar os secrets do Colab
import google_auth_oauthlib.flow
import google.auth.transport.requests
# Explicitly import auth from google.colab
from google.colab import auth


# Escopos necessários para acessar o Google Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar','https://www.googleapis.com/auth/calendar.events']

# Define TOKEN_FILE in a broader scope
TOKEN_FILE = 'token.json'

def get_google_credentials():
    """Obtém credenciais do Google Calendar usando Colab Secrets e Google Colab auth."""
    creds = None

    # O arquivo token.json salva os tokens de acesso e atualização do usuário.
    # Tenta carregar as credenciais de um token salvo anteriormente
    # Only attempt to load if the file exists
    if os.path.exists(TOKEN_FILE):
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
            print(f"Token carregado de {TOKEN_FILE}.")
            # Check if the loaded credentials are valid and have the correct scopes
            if creds and creds.valid and creds.has_scopes(SCOPES):
                 print("Token carregado é válido e tem escopos corretos.")
                 return creds # Return valid credentials
            else:
                 print("Token carregado é inválido ou não tem escopos corretos. Forçando nova autenticação.")
                 creds = None # Force re-authentication
        except Exception as e:
            print(f"Erro ao carregar ou validar token salvo ({TOKEN_FILE}): {e}")
            creds = None # Force re-authentication

    # If no valid credentials were loaded from the token file, or if it failed/was insufficient,
    # initiate the authorization flow using google.colab.auth
    if not creds or not creds.valid or not creds.has_scopes(SCOPES):
        print("Nenhuma credencial válida com os escopos necessários encontrada. Iniciando fluxo de autorização do Colab...")
        try:
            # Use google.colab.auth for authentication in Colab
            # This will provide an authorization URL and handle the redirect
            auth.authenticate_user(SCOPES) # Pass the list of scopes
            # After successful authentication, authenticate_user sets the default credentials
            # which can be obtained using google.auth.default()
            creds, project = google.auth.default()
            print("Fluxo de autorização do Colab concluído com sucesso.")

        except Exception as e:
            print(f"Ocorreu um erro durante o fluxo de autorização do Colab: {e}")
            return None # Retorna None em caso de erros


        # Salva as credenciais (incluindo o refresh token) para a próxima execução, se bem-sucedido
        # Note: In Colab, saving to a local file might not be persistent unless
        # you save to Google Drive or use Colab's built-in file management.
        if creds and creds.valid and creds.has_scopes(SCOPES):
            try:
                with open(TOKEN_FILE, 'w') as token:
                    token.write(creds.to_json())
                print(f"Novas credenciais salvas em {TOKEN_FILE}.")
            except Exception as e:
                print(f"Erro ao salvar token: {e}")
        else:
             print("As novas credenciais obtidas após o fluxo de autorização não contêm os escopos necessários.")
             return None # Return None if the newly obtained credentials also lack scopes


    return creds # Retorna as credenciais obtidas ou None se falhou


def agendar_vacina(nome_vacina, data_vacina_str):
    """Agenda um evento no Google Calendar."""

    # Chama a função para obter as credenciais
    creds = get_google_credentials()

    if not creds:
        print("Não foi possível obter credenciais. Não é possível agendar o evento no Google Calendar.")
        return

    try:
        # Constrói o serviço do Google Calendar
        service = build('calendar', 'v3', credentials=creds)

        # Formata a data para o formato do Google Calendar (YYYY-MM-DD ou dateTime)
        try:
            data_obj = datetime.datetime.strptime(data_vacina_str, '%Y-%m-%d')
            # Configura o evento para durar 1 hora no início do dia especificado
            start_time = data_obj.isoformat() + '+00:00' # Adiciona timezone UTC
            end_time = (data_obj + datetime.timedelta(hours=1)).isoformat() + '+00:00'
            # Se preferir um evento de dia inteiro, descomente as linhas abaixo e comente as acima:
            # start_dict = {'date': data_vacina_str}
            # end_dict = {'date': (data_obj + datetime.timedelta(days=1)).strftime('%Y-%m-%d')} # Evento de dia inteiro termina no início do dia seguinte

        except ValueError:
            print("Formato de data inválido. Use YYYY-MM-DD.")
            return

        event = {
            'summary': f'Vacina: {nome_vacina}',
            'start': {
                'dateTime': start_time, # Use 'date' aqui se for evento de dia inteiro
                'timeZone': 'America/Sao_Paulo', # Ajuste para o seu timezone
            },
            'end': {
                'dateTime': end_time, # Use 'date' aqui se for evento de dia inteiro
                'timeZone': 'America/Sao_Paulo',# Ajuste para o seu timezone
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 30}, # Lembrete 30 minutos antes
                ],
            },
        }

        # Insere o evento no calendário principal
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        print('Evento criado no Google Calendar: %s' % (created_event.get('htmlLink')))

    except Exception as e:
        print(f'Ocorreu um erro ao agendar no Google Calendar: {e}')

# Moved the function outside assistente_virtual and corrected indentation
def fornecer_frase_motivacional():
   try:
      frases_df = pd.read_csv('frases_motivacionais')
      frase = frases_df.sample(n=1)['Frases'].values[0]
      return frase # Return the phrase
   except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return 'Você está fazendo um belo trabalho como pai/mãe' # Return a default phrase

def assistente_virtual():

    display(HTML("<h2 style='color: #FF6347;'>👋 Olá, eu sou sua rede de apoio virtual e estou aqui para te ajudar! 🌟</h2>"))
    idade = input("Quantos meses tem o seu bebê?🍼")

    try:
        idade_bebe = int(idade)
        if idade_bebe <= 0:
            display(HTML("<p style='color: #4682B4;'>❗ Idade inválida. Por favor, insira uma idade maior que 0.</p>"))
        else:
            display(HTML(f"<h2 style='color: #4682B4;'>🎉 Seu bebê tem {idade_bebe} meses! 🎉</h2>"))

            # >>> SEU CÓDIGO PARA EXIBIR INFORMAÇÕES DE CUIDADOS COM O BEBÊ BASEADO NA IDADE <<<
            # Faixa etária de 0 a 2 meses
            if 0 <= idade_bebe <= 2:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 0 a 2 meses** 🛏️</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê dorme entre 14-17 horas por dia.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Leite materno ou fórmula a cada 2-3 horas.</li></ul>"))
                display(HTML("<ul><li>🧸 Brincadeiras: Converse, cante e faça sons suaves.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Acompanhe a vacinação do seu bebê.</li></ul>"))

            # Faixa etária de 3 a 4 meses
            elif 3 <= idade_bebe <= 4:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 3 a 4 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê dorme entre 12-16 horas por dia.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Introdução alimentar com papinhas.</li></ul>"))
                display(HTML("<ul><li>🧸 Brincadeiras: Brinquedos coloridos ajudam no desenvolvimento.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Lembre-se de seguir o calendário de vacinas.</li></ul>"))

            # Faixa etária de 5 a 6 meses
            elif 5 <= idade_bebe <= 6:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 5 a 6 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 12-16 horas por dia.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Introdução alimentar com papinhas de legumes e frutas.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode começar a virar de barriga para baixo.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Acompanhe as vacinas recomendadas.</li></ul>"))

            # Faixa etária de 7 a 8 meses
            elif 7 <= idade_bebe <= 8:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 7 a 8 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 12-14 horas por dia, com cochilos.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Continue com a introdução alimentar.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode começar a engatinhar e explorar o ambiente.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Verifique o calendário de vacinas.</li></ul>"))

            # Faixa etária de 9 a 10 meses
            elif 9 <= idade_bebe <= 10:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 9 a 10 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 12-14 horas por dia.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Alimentos mais sólidos e variados.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode já estar engatinhando ou até começando a andar com apoio.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Lembre-se de manter a vacinação em dia.</li></ul>"))

            # Faixa etária de 11 a 12 meses
            elif 11 <= idade_bebe <= 12:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 11 a 12 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 12-14 horas por dia.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: Ofereça alimentos mais sólidos e variados.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode começar a dar os primeiros passos ou a balbuciar palavras.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Verifique com o pediatra as vacinas recomendadas.</li></ul>"))

            # Faixa etária de 13 a 18 meses
            elif 13 <= idade_bebe <= 18:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 13 a 18 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 11-14 horas, incluindo cochilos.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: A alimentação deve ser bem variada e mais sólida.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode começar a andar sem apoio e a falar algumas palavras.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Mantenha o calendário de vacinas em dia.</li></ul>"))

            # Faixa etária de 19 a 24 meses
            elif 19 <= idade_bebe <= 24:
                display(HTML("<h3 style='color: #4682B4;'>**Cuidados com Bebê de 19 a 24 meses** 🧸</h3>"))
                display(HTML("<ul><li>🌙 Sono: O bebê pode dormir entre 11-12 horas, com cochilos mais curtos.</li></ul>"))
                display(HTML("<ul><li>🥛 Alimentação: A alimentação deve ser bem balanceada e diversificada.</li></ul>"))
                display(HTML("<ul><li>🧸 Desenvolvimento: O bebê pode já andar e começar a falar frases simples.</li></ul>"))
                display(HTML("<ul><li>💉 Vacinas: Consulte o pediatra para as vacinas e check-ups.</li></ul>"))

            # --- Lógica para Vacinas e Agendamento ---
            tem_vacina = False
            nome_vacina_recomendada = ""
            if idade_bebe == 0: # Ao nascer
                nome_vacina_recomendada = "BCG (dose única) e Hepatite B (1ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 1:
                 nome_vacina_recomendada = "Hepatite B (2ª dose)"
                 frase_motivacional = fornecer_frase_motivacional()
                 display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                 tem_vacina = True

            elif idade_bebe == 2:
                nome_vacina_recomendada = "Pentavalente (DTP/Hib/Hep B - 1ª dose), VIP (Poliomielite inativada - 1ª dose), Pneumocócica 10V (1ª dose), Rotavírus (1ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 3:
                nome_vacina_recomendada = "Meningocócica C (1ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 4:
                nome_vacina_recomendada = "Pentavalente (DTP/Hib/Hep B - 2ª dose), VIP (Poliomielite inativada - 2ª dose), Pneumocócica 10V (2ª dose), Rotavírus (2ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 5:
                nome_vacina_recomendada = "Meningocócica C (2ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 6:
                nome_vacina_recomendada = "Covid19, Pentavalente (DTP/Hib/Hep B - 3ª dose), VIP (Poliomielite inativada - 3ª dose), Hepatite B (3ª dose - se o esquema com a Penta não for aplicado no mesmo momento)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 7:
                nome_vacina_recomendada = "Covid19(2ª dose)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True
            elif idade_bebe == 9:
                nome_vacina_recomendada = "Febre Amarela (dose inicial - dependendo da área de residência)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 12: # 1 ano
                nome_vacina_recomendada = "Tríplice Viral (Sarampo, Caxumba, Rubéola - 1ª dose), Pneumocócica 10V (Reforço), Meningocócica C (Reforço)"
                frase_motivacional = fornecer_frase_motivacional()
                display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                tem_vacina = True

            elif idade_bebe == 15: # 1 ano e 3 meses
                 nome_vacina_recomendada = "DTP (Tríplice bacteriana - 1º reforço), VOP (Poliomielite oral - 1º reforço), Hepatite A (dose única), Tetra Viral (Sarampo, Caxumba, Rubéola, Varicela - dose única, substitui Tríplice Viral + Varicela)"
                 frase_motivacional = fornecer_frase_motivacional()
                 display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                 tem_vacina = True

            elif idade_bebe == 18: # 1 ano e 6 meses
                 nome_vacina_recomendada = "DTP (Tríplice bacteriana - 1º reforço - se não aplicada aos 15 meses), VOP (Poliomielite oral - 1º reforço - se não aplicada aos 15 meses)"
                 frase_motivacional = fornecer_frase_motivacional()
                 display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                 tem_vacina = True

            elif idade_bebe >= 19 and idade_bebe <= 24: # Entre 19 e 24 meses (1 ano e 7 meses a 2 anos)
                 # Algumas vacinas que podem ser verificadas ou reforçadas nesta faixa, dependendo do esquema
                 # É bom reforçar a importância de seguir o cartão de vacinação.
                 nome_vacina_recomendada = "Verificar calendário e cartão de vacinação para possíveis reforços ou doses pendentes (Ex: Varicela, Hepatite A - se esquema diferente). A vacina da Gripe é anual a partir dos 6 meses."
                 frase_motivacional = fornecer_frase_motivacional()
                 display(HTML(f"<h3 style='color: #4682B4;'> {frase_motivacional}  </h2>"))
                 tem_vacina = True

            else:
                nome_vacina_recomendada = "No momento, não há vacinas de rotina recomendadas para esta idade no calendário básico, mas verifique o cartão de vacinação para doses de reforço ou vacinas especiais. A vacina da Gripe é anual a partir dos 6 meses."

            # Adicione mais condições para outras faixas etárias e vacinas recomendadas
            # Exemplo:
            # elif idade_bebe == 6:
            #     nome_vacina_recomendada = "Poliomielite (VIP), Pneumocócica 13 (conjugada), Rotavírus (humano)"
            #     tem_vacina = True

            if tem_vacina:
                display(HTML(f"<p style='color: #228B22;'>💉 Para {idade_bebe} meses, as vacinas recomendadas podem incluir: **{nome_vacina_recomendada}**.</p>"))
                agendar = input("Deseja agendar uma vacina no Google Calendar? (sim/não): ").lower()
                if agendar == 'sim':
                    nome_vacina_agendar = input("Digite o nome da vacina para agendar: ")
                    data_agendamento = input("Digite a data de agendamento (AAAA-MM-DD): ")
                    # Call the scheduling function
                    agendar_vacina(nome_vacina_agendar, data_agendamento)
                else:
                    display(HTML("<p>Ok, não vamos agendar agora.</p>"))
            # --- Fim da Lógica para Vacinas e Agendamento ---

            # Exibe as opções iniciais de ajuda para o usuário
            display(HTML("<h2 style='color: #FF6347;'>👋 Olá, eu sou sua rede de apoio virtual e estou aqui para te ajudar! 🌟</h2>"))
            display(HTML("<h3 style='color: #228B22;'>Como posso te ajudar hoje?</h3>"))

            # As opções do assistente virtual
            display(HTML("""
              <ul>
                <li><a href='https://link-para-video-experto.com' target='_blank' style='color: #4682B4;'>1. Chamada de vídeo com especialista online</a></li>
                <li><a href='https://link-para-catalogo-ebook.com' target='_blank' style='color: #4682B4;'>2. Catálogos de Ebook</a></li>
                <li>3. Tirar dúvida e desabafar - Vamos conversar! (digite sua dúvida abaixo)</li></ul>
             """))
              # Lógica para interação com o usuário caso ele escolha "Tirar dúvida e desabafar"
            while True:
                escolha = input("Digite o número da opção escolhida ou 'fim' para finalizar: ").strip()
                if escolha == "fim":
                        display(HTML("<p>👋 Ok, se precisar de mais ajuda, estarei por aqui para você! Até logo!</p>"))
                        break
                if escolha == "3":  # Opção para tirar dúvidas e desabafar
                      display(HTML("<h3 style='color: #228B22;'>**Estou aqui para ouvir e ajudar! Como posso te ajudar hoje?** 🧸</h3>"))
                      ajuda = input("Escreva sua dúvida ou algo que queira desabafar: ").strip()
                      print(f"Estou processando sua dúvida ou desabafo: {ajuda} 🌍")

                   # Aqui você pode adicionar uma lógica para responder com base na dúvida
                      resposta = chat.send_message(ajuda)
                      print("Resposta: ", resposta.text)

                elif escolha == "1":
                      print("Você escolheu a chamada de vídeo com especialista online. Você pode agendar clicando no link ")
                      display(HTML("""<ul> <li><a href='https://link-para-video-experto.com' target='_blank' style='color: #4682B4;'>1. Chamada de vídeo com especialista online</a></li>
                """))
                elif escolha == "2":
                      print("Você escolheu os catálogos de Ebook. Você pode acessar clicando no link .")
                      display(HTML("""<ul> <li><a href='https://link-para-catalogo-ebook.com' target='_blank' style='color: #4682B4;'>2. Catálogo de Ebook</a></li>
                """))
                else:
                      print("Opção inválida. Tente novamente!")

    except ValueError:
        display(HTML("<p style='color: #FF0000;'>❗ Entrada inválida. Por favor, insira um número para a idade do bebê.</p>"))
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")


# Chama a função principal para iniciar o assistente
assistente_virtual()

Quantos meses tem o seu bebê?🍼2


Deseja agendar uma vacina no Google Calendar? (sim/não): não


Digite o número da opção escolhida ou 'fim' para finalizar: 2
Você escolheu os catálogos de Ebook. Você pode acessar clicando no link .


KeyboardInterrupt: Interrupted by user

Digite o número da opção escolhida ou 'fim' para finalizar: fim
